1. Place a reference crystal on a hotplate directly and put a cover on the fume hood
2. Run this code
3. Input the reference area
4. Modify the camera angle or hotplate location so that the green circle matches with the edge of the hotplate
5. Start filming and wait for > 10 secs
6. Press Q to quit
7. The program shows a proper scale value
8. Change the camera_scale in systeminformation.py

In [ ]:
import os
import sys
import numpy as np
import cv2
import time
import datetime
from pyFCCG import (
    img_to_video,
    save_img,
    elapsed_time_text,
    get_time,
    SGs,
    notify,
    send_img,
    int_ask,
    img_check,
    check_camera_port
)
from scipy.stats import linregress
import traceback
from systeminformation import get_system_info
import pandas as pd
from matplotlib import pyplot as plt

# ---------------------------- SYSTEM OPERATING MODES ---------------------------------
SYSTEM_ID = 'AY1'
camera_scale, radius, camera_settings, RGB_threshold, pump_type, pump_port, channelcode = get_system_info(SYSTEM_ID)
camera_ID = check_camera_port()

# ---------------------------- PARAMETERS ---------------------------------
interval_for_shots = 1  # sec, time between shots and area calculation
total_duration_hrs = 5000 #total program duration in hours

ref_area = input('What is the area of your reference crystal? mm2?')
ref_area = float(ref_area)

####set parameters
# filming
ww, hh = int(1920), int(1080)

# Video information
checkpoint_for_video = 0 # sec, will be 0, 600, 1200, ... if the interval_for_video is 600
seconds_duration = float("%0.2f" % (float(total_duration_hrs) * 3600))
scale = float("%0.2f" % float(camera_scale))

# Area, Growth rate calculation
detection_mode = "AREA" # assume the biggest crystal is the seed crystal
# detection_mode = 'POSITION' # assume the nearest crystal to the crystal in previous image is the seed crystal

lower = np.array([0, 0, 50])  # B,G,R threshold to find orange regions
upper = np.array([255, 60, 255])

# others
font = cv2.FONT_HERSHEY_SIMPLEX

# list preparation
time_list = []
length_list = []
area_list = []

cap = img_check(camera_ID,camera_settings, r=radius)

## You will start infusion and imaging
while True:
    Switch = input("Start(0) or Quit(1)?\n")
    if Switch == "0":
        print("\nLet's get started\n")
        break
    if Switch == "1":
        print("\n Fine. Try again.")
        sys.exit(0)
    else:
        pass

# dismiss the too bright pictures for first 1.5 seconds
for k in range(3):
    ret, img = cap.read()
    time.sleep(0.5)

start_time = time.monotonic()
finish_time = start_time + seconds_duration

i = 0

print("Start Filming at " + str(datetime.datetime.now()) + "\n")
print("Working...\n")

# Error message will be send on Slack if it happens
try:
    ####in-situ growth start####
    while time.monotonic() < finish_time:
        round_start_time = time.monotonic()

        ##TAKE A PICTURE
        ret, im1 = cap.read()

        # get time
        realtime = get_time("hour", start_time)
        time_list.append(realtime)
        
        realtime_sec = get_time("second", start_time) # variable for repeating calculation, control, etc

        # print as 00h 00m 00s
        ela_time_path = elapsed_time_text(start_time)  # 00h00m00s, will be used for save_path_name
        ela_time = "Time: " + ela_time_path

        ##AREA CALCULATION
        # draw fill circle in white on black background as mask
        mask = np.zeros_like(im1)
        mask = cv2.circle(mask, (int(ww / 2), int(hh / 2)), radius, (255, 255, 255), -1)
        # apply mask to image
        im2 = cv2.bitwise_and(im1, mask)
        # prepare a mask to extract orange colors
        mask = cv2.inRange(im2, lower, upper)
        cons, hierarchy = cv2.findContours(
            mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        # area
        alist1 = []
        est_mass = 0
        # in case of there is no crystal
        if len(cons) == 0:
            area, seed_area = 0, 0
            alist1.append(area)
        # if there is a crystal
        else:
            for cnt in cons:
                area = cv2.contourArea(cnt) / scale
                alist1.append(area)

            if detection_mode == "AREA":
                seed_area = max(alist1)
                # save the center position
                try:
                    M = cv2.moments(cons[alist1.index(seed_area)])
                    cx_seed = M["m10"] / M["m00"]
                    cy_seed = M["m01"] / M["m00"]
                    detection_mode = "POSITION"
                except:
                    pass

            elif detection_mode == "POSITION":
                distance_list, M_list = [], []
                for cnt in cons:
                    M = cv2.moments(cnt)
                    M_list.append(M)
                    try:
                        cx = M["m10"] / M["m00"]
                        cy = M["m01"] / M["m00"]
                        distance = (cx_seed - cx) ** 2 + (cy_seed - cy) ** 2
                    except Exception:
                        distance = 10000000
                    distance_list.append(distance)
                seed_index = distance_list.index(min(distance_list))
                M = M_list[seed_index]
                cx_seed = M["m10"] / M["m00"]
                cy_seed = M["m01"] / M["m00"]
                seed_area = alist1[seed_index]

        area_list.append(float("{:.2f}".format(seed_area)))
        length = np.sqrt(area_list[-1])
       
        # make processed image
        im2 = cv2.drawContours(im2, cons, -1, (0, 150, 0), 2)
        im2 = cv2.drawContours(im2, cons, alist1.index(seed_area), (0, 255, 0), 2)
        area_text = "Area: " + str("{:.2f}".format(seed_area)) + " mm2"
        len_text = "Length: " + str("{:.2f}".format(length)) + " mm"
        cv2.putText(
            im2,
            ela_time,
            org=(780, 60),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )
        cv2.putText(
            im2,
            area_text,
            org=(610, 1040),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )

        cv2.putText(
            im2,
            len_text,
            org=(1010, 1040),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )


        im2_ = im2.copy()

        # prepare imshow for realtime observation
        total_time = "Total Time: " + str(total_duration_hrs).zfill(2) + " h"
        cry_num_text = "Number of crystals: " + str(len(cons))
       
        cv2.putText(
            im2,
            "---- Parameters ----",
            org=(60, 100),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )
        cv2.putText(
            im2,
            total_time,
            org=(60, 160),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )
        cv2.putText(
            im2,
            cry_num_text,
            org=(60, 220),
            fontFace=font,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
            lineType=cv2.LINE_AA,
        )
        
        cv2.namedWindow("img", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("img", 1500, int(1500 * im2.shape[0] / im2.shape[1]))
        cv2.imshow("img", im2)


        # add interval
        interval_time = time.monotonic() - round_start_time
        if interval_time >= interval_for_shots:
            pass
        elif interval_time < interval_for_shots:
            time.sleep(interval_for_shots - interval_time)

    
        if cv2.waitKey(20) & 0xFF == ord("q"):
            print("Break at " + str(datetime.datetime.now()) + "\n")
            cap.release()
            cv2.destroyAllWindows()
            break

        i += 1

except Exception as e:
    # When error occurs
    try:
        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print(e)
        pass
    print("Error at " + str(datetime.datetime.now()) + "\n" + traceback.format_exc())

area = sum(area_list)/len(area_list) * scale

plt.rcParams["figure.figsize"] = [5.0, 3.50]
plt.rcParams["figure.autolayout"] = True
data = pd.DataFrame({"Area": area_list})
data.boxplot()
for i, d in enumerate(data):
    y = data[d]
    x = np.random.normal(i + 1, 0.04, len(y))
    plt.scatter(x, y)
plt.show()

print('Detected area was ' + str(int(area)) + ' pixel\n')
print('Reference area was ' + str(ref_area) + ' mm2\n')

new_scale = area / ref_area

print('The current scale is ' + str('{:.2f}'.format(scale)) + ' pix/mm2\n')
print('Scale should be ' + str('{:.2f}'.format(new_scale)) + ' pix/mm2\n')